In [2]:
import json
import pandas as pd
import os
from os.path import exists

In [ ]:
# Define the directory to transform files within
# directory = os.getcwd()
directory = "C:/Users/ajarvinen/OneDrive - KPMG/Desktop/polar-user-data-export_e8293a83-b2e8-47b9-9545-ea2de41e79bb/skiing"

In [6]:
# Define the transformation function repeated to every json-file

def transformJson(filename):

    # Extract the time string
    time = filename[17:38]

    # Read json
    f = open(filename)
    data = json.load(f)

    # Check the type of the training
    sport = data['exercises'][0]['sport']

    if sport == "CYCLING" or sport == "CROSS-COUNTRY_SKIING":
        df_route = pd.json_normalize(data['exercises'][0]['samples']['recordedRoute'])
        df_heartRate = pd.json_normalize(data['exercises'][0]['samples']['heartRate'])
        df_speed = pd.json_normalize(data['exercises'][0]['samples']['speed'])
        df_distance = pd.json_normalize(data['exercises'][0]['samples']['distance'])

        # Create merged dataframe
        merged = pd.merge(df_heartRate,df_speed,on="dateTime")
        merged = pd.merge(merged,df_distance,on="dateTime")

        # Create key columns based on seconds (rounded values)
        s = df_route['dateTime'].astype(str)
        df_route['key'] = s.str[0:19]

        s = merged['dateTime'].astype(str)
        merged['key'] = s.str[0:19]

        # Merge all data - use outer join to preserve all data
        merged = pd.merge(merged,df_route,on="key",how="outer")

        # drop, rename, reorganize columns
        merged = merged.drop(['dateTime_y'], axis=1)
        merged = merged.rename(columns={"value_x": "heartRate", "value_y": "speed", "value" : "distance", "dateTime_x" : "dateTime"})
        merged = merged[['key','dateTime','longitude','latitude','altitude','heartRate','speed','distance']]

        # Write csv
        name = directory+"/"+sport+"_"+time+".csv"
        merged.to_csv(name,index=False)

In [ ]:
# Iterate through all files in the folder
for filename in os.listdir(directory):
    if filename.endswith(".json") and "training-session" in filename: 
        transformJson(filename)
        continue
    else:
        continue